In [87]:
import math
import os

import numpy as np
from keras import Sequential, Model
from keras.layers import Dense
from keras.optimizer_v2.adam import Adam

from src.dlla.hw import encode, dlla_hw
from src.dlla.wegener import make_mlp_wegener, binomial_test
from src.pollution.clock_jitter import clock_jitter
from src.pollution.gaussian_noise import gaussian_noise
from src.pollution.random_delay import random_delay
from src.trace_set.database import Database
from src.trace_set.pollution import Pollution, PollutionType
from src.trace_set.set_hw import TraceSetHW
from src.trace_set.transform import reduce_fixed_fixed, fixed_fixed
from src.trace_set.window import get_windows
from src.tvla.cri import tvla_cri
from src.tvla.tvla import Group

In [17]:
DB = Database.aisy
DB_RAW = Database.aisy_raw

In [18]:
RAW_TRACES = TraceSetHW(DB_RAW)

X_PROF_CXT, Y_PROF = RAW_TRACES.profile_states()
X_ATT_CXT, Y_ATT = RAW_TRACES.attack_states()

A_THIRD = round(X_PROF_CXT.shape[1] / 3)
SAMPLE_TRACE = X_PROF_CXT[0][A_THIRD:A_THIRD*2]

In [19]:
WINDOW, _ = get_windows(X_PROF_CXT, SAMPLE_TRACE)

In [20]:
X_PROF = X_PROF_CXT[:, WINDOW[0]:WINDOW[1]]
X_ATT = X_ATT_CXT[:, WINDOW[0]:WINDOW[1]]

X_PROF_CXT.shape, X_PROF.shape, Y_PROF.shape

((6667, 21996), (6667, 7332), (6667,))

In [21]:
JITTER_PARAMS = np.arange(0, 42, 2)

def apply_jitter(params):
    for param in params:
        pollution = Pollution(PollutionType.jitter, param)
        out = TraceSetHW(DB, pollution)

        if not os.path.exists(out.path):
            x = clock_jitter(X_PROF_CXT, WINDOW, param)
            x_att = clock_jitter(X_ATT_CXT, WINDOW, param)

            out.create(x, Y_PROF, x_att, Y_ATT)

            x2, y2 = fixed_fixed(x, Y_PROF)
            a, b = x2[~y2], x2[y2]
            order = 2
            _, tvla_p = Group(a, order, True).t_test(Group(b, order, True), order)
            print(f"Clock jitter ({param}): min-p: ({min(tvla_p)}).")

apply_jitter(JITTER_PARAMS)

Clock jitter (4):   0%|          | 10/6667 [00:00<01:12, 91.56it/s]

Clock jitter (2): min-p: (8.800726234515573e-270).


Clock jitter (6):   0%|          | 13/6667 [00:00<00:54, 122.31it/s]

Clock jitter (4): min-p: (1.5948878621373198e-206).


Clock jitter (8):   0%|          | 15/6667 [00:00<00:45, 146.12it/s]

...

Clock jitter (36): min-p: (9.675016489785921e-148).


Clock jitter (40):   1%|          | 55/6667 [00:00<00:12, 541.93it/s]

Clock jitter (38): min-p: (1.438664253093877e-172).


Computing Central Moments: 100%|██████████| 4/4 [00:00<00:00,  6.17it/s]

Clock jitter (40): min-p: (2.923095471831935e-155).


In [133]:
GAUSS_PARAMS = np.arange(0, 42, 2)

def apply_gauss(params):
    for param in params:
        pollution = Pollution(PollutionType.gauss, param)
        out = TraceSetHW(DB, pollution)

        if not os.path.exists(out.path):
            x = gaussian_noise(X_PROF, param)
            x_att = gaussian_noise(X_ATT, param)

            out.create(x, Y_PROF, x_att, Y_ATT)

            x2, y2 = fixed_fixed(x, Y_PROF)
            a, b = x2[~y2], x2[y2]
            order = 2
            _, tvla_p = Group(a, order, True).t_test(Group(b, order, True), order)
            print(f"Gaussian noise ({param}): min-p: ({min(tvla_p)}).")

apply_gauss(GAUSS_PARAMS)

In [23]:
DELAY_PARAMS = np.arange(0, .80, .05)
A = 5
B = 3
DELAY_AMP = 10

def apply_delay(params):
    for param in params:
        pollution = Pollution(PollutionType.delay, param)
        out = TraceSetHW(DB, pollution)

        if not os.path.exists(out.path):
            x = random_delay(X_PROF, A, B, DELAY_AMP, param)
            x_att = random_delay(X_ATT, A, B, DELAY_AMP, param)

            out.create(x, Y_PROF, x_att, Y_ATT)

            x2, y2 = fixed_fixed(x, Y_PROF)
            a, b = x2[~y2], x2[y2]
            order = 2
            _, tvla_p = Group(a, order, True).t_test(Group(b, order, True), order)
            print(f"Random delay ({param}): min-p: ({min(tvla_p)}).")

apply_delay(DELAY_PARAMS)

Random delay (0.05):   0%|          | 32/6667 [00:00<00:21, 312.28it/s]

Random delay (0.0): min-p: (3.485803269518708e-250).


Random delay (0.1):   0%|          | 27/6667 [00:00<00:25, 264.33it/s]

Random delay (0.05): min-p: (3.728709520608966e-258).


Random delay (0.15000000000000002):   0%|          | 25/6667 [00:00<00:27, 243.58it/s]

...

Random delay (0.65): min-p: (2.621909815897095e-115).


Random delay (0.75):   0%|          | 18/6667 [00:00<00:37, 176.05it/s]

Random delay (0.7000000000000001): min-p: (7.395822289676536e-126).


Computing Central Moments: 100%|██████████| 4/4 [00:00<00:00,  6.12it/s]

Random delay (0.75): min-p: (4.3594052379937256e-129).


In [24]:
GAUSS_PARAMS = np.arange(0, 32, 2)

### TVLA vs. DL-LA

In [57]:
ORDER = 2

def store_results(database: Database, num_traces: int, method: str, pollution: Pollution, p):
    file_name = f"results_{database.name}.csv"
    with open(file_name, 'a') as f:
        f.write(f"{method};{num_traces};{pollution.type.name};{pollution.parameter};{p}\n")

In [136]:
def prepare_traces_dl(x, y, x_att, y_att):
    """
    Normalizes the traces, one-hot encodes the labels.
    Returns profiling traces, labels and attack traces, labels.
    """
    prof_mean, prof_std = x.mean(axis=0), x.std(axis=0)
    norm_x = (x - prof_mean) / prof_std
    norm_x_att = (x_att - prof_mean) / prof_std

    return norm_x, encode(y), norm_x_att, encode(y_att)


def build_mlp(x, y, params):
    mdl = Sequential()
    mdl.add(Dense(100, activation=params['activation'], input_shape=(x.shape[1],)))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(9, activation='softmax'))

    mdl.compile(optimizer=params['optimizer'], loss=params['losses'], metrics=['accuracy'])

    out = mdl.fit(x, y, shuffle=True, batch_size=params['batch_size'], epochs=params['epochs'], verbose=False)

    return out, mdl


def make_mlp(x, y):
    return build_mlp(x, y, {
        'activation': 'relu',
        'optimizer': Adam(learning_rate=0.001),
        'losses': 'categorical_crossentropy',
        'batch_size': 150,
        'epochs': 5
    })[1]

def wegener_p(mdl: Model, x_att: np.ndarray, y_att: np.ndarray):
    predictions = mdl.predict(x_att).argmax(axis=1)
    labels = y_att.argmax(axis=1)

    correct = np.sum(predictions == labels)
    total = len(predictions)
    # print(total, correct)

    return binomial_test(total, correct)

LIMIT_PROF = 400
LIMIT_ATT = round(.25 * LIMIT_PROF)

def la_benchmark(db: Database, pollution_type: PollutionType, params, limits=(LIMIT_PROF, LIMIT_ATT)):
    for param in params:
        pollution = Pollution(pollution_type, param)
        print("Load traces         ", end="\r")
        trace_set = TraceSetHW(db, pollution, limits)

        if os.path.exists(trace_set.path):
            print("Prepare traces (1/2)", end="\r")
            x9_all, y9, x9_att_all, y9_att = prepare_traces_dl(*trace_set.profile(), *trace_set.attack())
            x9, x9_att = x9_all, x9_att_all

            print("Prepare traces (2/2)", end="\r")
            (x2, y2), (x2_att, y2_att) = reduce_fixed_fixed(x9, y9), reduce_fixed_fixed(x9_att, y9_att)

            try:
                print("Make model (1/2)    ", end="\r")
                mdl9 = make_mlp(x9, y9)
                print("Make model (2/2)    ", end="\r")
                mdl2 = make_mlp_wegener(x2, y2, False)

                print("Predict             ", end="\r")
                dlla9_p, _ = dlla_hw(mdl9, x9_att, y9_att)
                dlla2_p = wegener_p(mdl2, x2_att, y2_att)

                if math.isnan(dlla9_p):
                    dlla9_p = 1.
            except ValueError:
                dlla9_p, dlla2_p = [1.] * 2

            print("TVLA                ", end="\r")
            tvla_ps = np.min(tvla_cri(trace_set, ORDER), axis=1)

            print(f"{pollution_type} ({param}). TVLA ({tvla_ps}). DLLA9 ({dlla9_p}). DLLA2 ({dlla2_p})", end="\r")

            num_traces = LIMIT_PROF + LIMIT_ATT
            for order, p in enumerate(tvla_ps):
                if order > 0:
                    store_results(db, num_traces, f"cri_tvla_{order}", pollution, p)

            store_results(db, num_traces, "dlla9", pollution, dlla9_p)
            store_results(db, num_traces, "dlla2", pollution, dlla2_p)

        print()

la_benchmark(DB, PollutionType.gauss, [100])

PollutionType.gauss (100). TVLA ([1.00000000e+00 3.80051193e-08 7.64223294e-05]). DLLA9 (2.535953351903791e-10). DLLA2 (9.650597348809242e-06)


In [76]:
np.logspace(2, 3.5, 25)

array([ 100.        ,  115.47819847,  133.35214322,  153.99265261,
        177.827941  ,  205.35250265,  237.13737057,  273.84196343,
        316.22776602,  365.17412725,  421.69650343,  486.96752517,
        562.34132519,  649.38163158,  749.89420933,  865.96432336,
       1000.        , 1154.78198469, 1333.52143216, 1539.92652606,
       1778.27941004, 2053.52502646, 2371.37370566, 2738.41963426,
       3162.27766017])

In [137]:
while True:
    # la_benchmark(DB, PollutionType.jitter, JITTER_PARAMS)
    # la_benchmark(DB, PollutionType.delay, DELAY_PARAMS)
    la_benchmark(DB, PollutionType.gauss, GAUSS_PARAMS)


PollutionType.gauss (0). TVLA ([1.00000000e+00 1.41246116e-49 5.21769271e-39]). DLLA9 (1.7709021737555773e-13). DLLA2 (1.2780074030160904e-06)
PollutionType.gauss (8). TVLA ([1.00000000e+00 3.12335667e-55 6.40748281e-31]). DLLA9 (1.1224455218325548e-11). DLLA2 (2.3283064365386963e-10)
PollutionType.gauss (16). TVLA ([1.0000000e+00 2.5626615e-44 4.1804020e-17]). DLLA9 (6.955252149909383e-14). DLLA2 (5.653710104525089e-05)
PollutionType.gauss (24). TVLA ([1.00000000e+00 9.23307097e-33 1.44097448e-16]). DLLA9 (4.9848658198014004e-12). DLLA2 (9.650597348809242e-06)
PollutionType.gauss (32). TVLA ([1.00000000e+00 6.61111838e-26 3.06818523e-13]). DLLA9 (1.3782820870879117e-10). DLLA2 (0.0002675263676792383)
PollutionType.gauss (40). TVLA ([1.00000000e+00 4.70655441e-24 3.30163966e-08]). DLLA9 (2.4649194759628234e-09). DLLA2 (1.2780074030160904e-06)
PollutionType.gauss (48). TVLA ([1.00000000e+00 4.79622992e-22 1.00137921e-06]). DLLA9 (1.0483737777832589e-11). DLLA2 (0.0002675263676792383)
Po

KeyboardInterrupt: 